На этой неделе вам предстоит попробовать добавить в вашу регрессионную модель дополнительные признаки.

1. Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте Qmay — качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.
2. Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.
3. Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16∣∣y^rT|T+i−yrT+i∣∣.
4. Убедитесь, что среднее качество прогнозов увеличилось.
5. Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: geoID, histEndDay, histEndHour, step, y
6. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.
7. Загрузите ноутбук в форму.

In [1]:
# Порядок расчетов оставим прежним, добавим только несколько дополнительных признаков
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from datetime import timedelta

In [ ]:
# количество пассажиров
# праздничный или выходной день
# данные о погоде

In [2]:
raw_data = pd.read_csv('data/df_result2.csv')
raw_data.set_index('date', inplace=True)
raw_data.sort_index(inplace=True)

raw_data.head()

,start_region,trips_count
date,,
2015-01-01 00:00:00,2500,0
2015-01-01 00:00:00,1663,0
2015-01-01 00:00:00,1664,0
2015-01-01 00:00:00,1665,0
2015-01-01 00:00:00,1666,0


In [3]:
# данные хранились в двух столбцах, развернем в сводную таблицу по ячейкам
raw_data2 = pd.pivot_table(raw_data, index='date', columns='start_region', values='trips_count', aggfunc=sum)
raw_data2.index = pd.to_datetime(raw_data2.index)
raw_data2.head()

start_region,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data_may = raw_data2.loc['2016-05-01 00:00:00':'2016-05-31 23:00:00', :]
(data_may.mean()>5).sum()

102

In [5]:
# список интересующих ячеек
r_list = data_may.loc[:, data_may.mean()>5].columns

In [6]:
filtered_data = raw_data2.loc['2016-01-01 00:00:00':, r_list]
filtered_data.head(2)

start_region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29


In [ ]:
# генерация временных признаков

In [7]:
data = filtered_data.copy()

# месяц, день месяца, день недели, чаc
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['day of week'] = data.index.dayofweek
data['hour'] = data.index.hour

# годовые, месячные и суточные признаки Фурье
K = 15
year_period = 8766 #365.2424*24 - средняя продолжительность года в часах
week_period = 168
day_period = 24
for i in range(1, K+1):
    data['y_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
    data['y_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
for i in range(1, K+1):    
    data['w_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
    data['w_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
for i in range(1, K+1):    
    data['d_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)
    data['d_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)

data.head()

start_region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,d_с_11,d_s_11,d_с_12,d_s_12,d_с_13,d_s_13,d_с_14,d_s_14,d_с_15,d_s_15
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,-0.965926,0.258819,-1.0,1.224647e-16,-0.965926,-0.258819,-8.660254e-01,-0.500000,-7.071068e-01,-7.071068e-01
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,0.866025,-0.500000,1.0,-2.449294e-16,0.866025,0.500000,5.000000e-01,0.866025,1.194340e-15,1.000000e+00
2016-01-01 02:00:00,90,146,23,110,393,425,313,366,377,138,...,-0.707107,0.707107,-1.0,3.673940e-16,-0.707107,-0.707107,-4.286264e-16,-1.000000,7.071068e-01,-7.071068e-01
2016-01-01 03:00:00,32,87,16,62,252,399,324,309,327,166,...,0.500000,-0.866025,1.0,-4.898587e-16,0.500000,0.866025,-5.000000e-01,0.866025,-1.000000e+00,2.388680e-15
2016-01-01 04:00:00,24,43,10,53,145,254,264,333,318,145,...,-0.258819,0.965926,-1.0,2.388680e-15,-0.258819,-0.965926,8.660254e-01,-0.500000,7.071068e-01,7.071068e-01


In [ ]:
# Сгенерируем тригонометрические признаки на основе расчетов 3-4 недели
# Сгенерируем оптимальный набор тригонометрических признаков
for i in range(1, 16):        
    data['s_'+str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/168)
    data['c_'+str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/168)

In [9]:
# **Создание регрессионных моделей для каждого региона**
# пока только на основе тригонометрических и временных
# - добавить праздничные дни

In [10]:
%%time
import warnings
warnings.filterwarnings('ignore')

linear_models_dict = {}

X_val_dict = {}
y_val_dict = {}
y_val_pred_dict = {}

X_test_dict = {}
y_test_dict = {}
y_test_pred_dict = {}

for id_name in filtered_data.columns:
    print(id_name,end=' ')
    drop_list = list(filtered_data.columns)
    drop_list.remove(id_name)
    named_data = data.copy().drop(drop_list, axis = 1) #Избавляемся от лишних id на итерации
    
    #Добавляем shift признаки
    named_data['shift_1'] = named_data[id_name].shift(1)
    named_data['shift_2'] = named_data[id_name].shift(2)
    named_data['shift_3'] = named_data[id_name].shift(3)
    named_data['shift_4'] = named_data[id_name].shift(4)
    named_data['shift_5'] = named_data[id_name].shift(5)
    named_data['shift_6'] = named_data[id_name].shift(6)
    named_data['shift_24'] = named_data[id_name].shift(24)
    named_data['shift_48'] = named_data[id_name].shift(48)
    
    named_data = named_data.iloc[48:]
    
    # Разбиение данных
    predict_may_start = '2016-04-30 23:00'
    predict_may_stop = '2016-05-31 23:00'
    precit_june_stop = '2016-06-30 23:00' 
    
    X_train = named_data.loc[:predict_may_start].drop(id_name, axis = 1)
    y_train = named_data.loc[:predict_may_start][id_name]
    
    X_val = named_data.loc[predict_may_start:predict_may_stop].drop(id_name, axis = 1)
    y_val = named_data.loc[predict_may_start:predict_may_stop][id_name]
    
    X_test = named_data.loc[predict_may_stop:].drop(id_name, axis = 1)
    y_test = named_data.loc[predict_may_stop:][id_name]
    
    #Строим Ridge модель с подбором гиперпараметра
    model = linear_model.Ridge()
    param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
    clf = GridSearchCV(model, param_grid, cv = 5)
    best_model = clf.fit(X_train, y_train)
    
    # Сохраним результаты разбиения данных, моделей и их прогнозы по словарям. Ключлм будет являться id региона
    linear_models_dict[id_name] = best_model
    X_val_dict[id_name] = X_val
    y_val_dict[id_name] = y_val
    y_val_pred_dict[id_name] = best_model.predict(X_val)
    X_test_dict[id_name] = X_test
    y_test_dict[id_name] = y_test
    y_test_pred_dict[id_name] = best_model.predict(X_test)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 Wall time: 3min 5s


In [11]:
# Анализ предсказаний за май
may_time_range = pd.date_range('2016-04-30 23:00', '2016-05-31 17:00', freq = 'H')

In [13]:
%%time
Q_may_f = 0
for key in filtered_data.columns:
    print(key,end=' ')
    for date in may_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_val_pred_dict.get(key), index = y_val_dict.get(key).index, columns = ['val'])
            real_values = y_val_dict.get(key)
            Q_may_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_may = float(Q_may_f/(102*739*6))
print() 
print('Q_may: ', Q_may)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_may:  17.549206257849523
Wall time: 2min 12s


In [14]:
#Добавляем праздничные дни
# Celebrations
new_data = data.copy()

new_data['pre_new_year'] = [1 if date.month == 12 and date.day == 31 else 0 for date in new_data.index]
new_data['new_year'] = [1 if date.month == 1 and date.day == 1 else 0 for date in new_data.index]
new_data['ind_day'] = [1 if date.month == 7 and date.day == 4 else 0 for date in new_data.index]
new_data['valentine_day'] = [1 if date.month == 2 and date.day == 14 else 0 for date in new_data.index]
new_data['inag_day'] = [1 if date.month == 1 and date.day == 20 else 0 for date in new_data.index]
new_data['veter_day'] = [1 if date.month == 11 and date.day == 11 else 0 for date in new_data.index]
new_data['9_11_day'] = [1 if date.month == 9 and date.day == 11 else 0 for date in new_data.index]
new_data['xmas_day'] = [1 if date.month == 12 and date.day == 25 else 0 for date in new_data.index]
new_data['lut_day'] = [1 if date.month == 1 and ((date.year == 2014 and date.day == 20) or \
                                                    (date.year == 2015 and date.day == 19) or\
                                                    (date.year == 2016 and date.day == 18))
                        else 0 for date in new_data.index]
new_data['labor_day'] = [1 if date.month == 9 and ((date.year == 2014 and date.day == 1) or \
                                                    (date.year == 2015 and date.day == 7) or\
                                                    (date.year == 2016 and date.day == 5))
                        else 0 for date in new_data.index]
new_data['columb_day'] = [1 if date.month == 10 and ((date.year == 2014 and date.day == 13) or \
                                                    (date.year == 2015 and date.day == 12) or\
                                                    (date.year == 2016 and date.day == 10))
                            else 0 for date in new_data.index]
new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                    (date.year == 2015 and date.day == 16) or\
                                                    (date.year == 2016 and date.day == 15))
                            else 0 for date in new_data.index]
new_data['president_day'] = [1 if date.month == 2 and ((date.year == 2014 and date.day == 17) or \
                                                    (date.year == 2015 and date.day == 16) or\
                                                    (date.year == 2016 and date.day == 15))
                            else 0 for date in new_data.index]
new_data['memm_day'] = [1 if date.month == 5 and ((date.year == 2014 and date.day == 26) or \
                                                    (date.year == 2015 and date.day == 25) or\
                                                    (date.year == 2016 and date.day == 30))
                            else 0 for date in new_data.index]  #good feature
new_data['flags_day'] = [1 if date.month == 6 and date.day == 14 else 0 for date in new_data.index]

new_data.head()

start_region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,inag_day,veter_day,9_11_day,xmas_day,lut_day,labor_day,columb_day,president_day,memm_day,flags_day
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 02:00:00,90,146,23,110,393,425,313,366,377,138,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 03:00:00,32,87,16,62,252,399,324,309,327,166,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 04:00:00,24,43,10,53,145,254,264,333,318,145,...,0,0,0,0,0,0,0,0,0,0


In [15]:
%%time
import warnings
warnings.filterwarnings('ignore')

linear_models_dict = {}

X_val_dict = {}
y_val_dict = {}
y_val_pred_dict = {}

X_test_dict = {}
y_test_dict = {}
y_test_pred_dict = {}

for id_name in filtered_data.columns:
    print(id_name,end=' ')
    drop_list = list(filtered_data.columns)
    drop_list.remove(id_name)
    named_data = new_data.copy().drop(drop_list, axis = 1) #Избавляемся от лишних id на итерации
    
    #Добавляем shift признаки
    named_data['shift_1'] = named_data[id_name].shift(1)
    named_data['shift_2'] = named_data[id_name].shift(2)
    named_data['shift_3'] = named_data[id_name].shift(3)
    named_data['shift_4'] = named_data[id_name].shift(4)
    named_data['shift_5'] = named_data[id_name].shift(5)
    named_data['shift_6'] = named_data[id_name].shift(6)
    named_data['shift_24'] = named_data[id_name].shift(24)
    named_data['shift_48'] = named_data[id_name].shift(48)
    
    named_data = named_data.iloc[48:]
    
    # Разбиение данных
    predict_may_start = '2016-04-30 23:00'
    predict_may_stop = '2016-05-31 23:00'
    precit_june_stop = '2016-06-30 23:00' 
    
    X_train = named_data.loc[:predict_may_start].drop(id_name, axis = 1)
    y_train = named_data.loc[:predict_may_start][id_name]
    
    X_val = named_data.loc[predict_may_start:predict_may_stop].drop(id_name, axis = 1)
    y_val = named_data.loc[predict_may_start:predict_may_stop][id_name]
    
    X_test = named_data.loc[predict_may_stop:].drop(id_name, axis = 1)
    y_test = named_data.loc[predict_may_stop:][id_name]
    
    #Строим Ridge модель с подбором гиперпараметра
    model = linear_model.Ridge()
    param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
    clf = GridSearchCV(model, param_grid, cv = 5)
    best_model = clf.fit(X_train, y_train)
    
    # Сохраним результаты разбиения данных, моделей и их прогнозы по словарям. Ключлм будет являться id региона
    linear_models_dict[id_name] = best_model
    X_val_dict[id_name] = X_val
    y_val_dict[id_name] = y_val
    y_val_pred_dict[id_name] = best_model.predict(X_val)
    X_test_dict[id_name] = X_test
    y_test_dict[id_name] = y_test
    y_test_pred_dict[id_name] = best_model.predict(X_test)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 Wall time: 3min 15s


In [16]:
%%time
Q_may_f = 0
for key in filtered_data.columns:
    print(key,end=' ')
    for date in may_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_val_pred_dict.get(key), index = y_val_dict.get(key).index, columns = ['val'])
            real_values = y_val_dict.get(key)
            Q_may_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_may = float(Q_may_f/(102*739*6))
print() 
print('Q_may: ', Q_may)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_may:  17.550623401982023
Wall time: 2min 10s


In [ ]:
# Далее будем добавлять скользящие суммы за 6, 24, 168 часов

In [24]:
%%time
import warnings
warnings.filterwarnings('ignore')

linear_models_dict = {}

X_val_dict = {}
y_val_dict = {}
y_val_pred_dict = {}

X_test_dict = {}
y_test_dict = {}
y_test_pred_dict = {}

for id_name in filtered_data.columns:
    print(id_name,end=' ')
    drop_list = list(filtered_data.columns)
    drop_list.remove(id_name)
    named_data = new_data.copy().drop(drop_list, axis = 1) #Избавляемся от лишних id на итерации
    
    #Добавляем shift признаки
    named_data['shift_1'] = named_data[id_name].shift(1)
    named_data['shift_2'] = named_data[id_name].shift(2)
    named_data['shift_3'] = named_data[id_name].shift(3)
    named_data['shift_4'] = named_data[id_name].shift(4)
    named_data['shift_5'] = named_data[id_name].shift(5)
    named_data['shift_6'] = named_data[id_name].shift(6)
    named_data['shift_24'] = named_data[id_name].shift(24)
    named_data['shift_48'] = named_data[id_name].shift(48)

    # 12-hours 
    named_data['half_day_sum'] = named_data[id_name].rolling(12).sum().fillna(0)
    # 6-hours
    named_data['quat_day_sum'] = named_data[id_name].rolling(6).sum().fillna(0)
    # 24-hours
    named_data['day_sum'] = named_data[id_name].rolling(24).sum().fillna(0)
    # 168-hours
    named_data['week_sum'] = named_data[id_name].rolling(168).sum().fillna(0) 
    
    named_data = named_data.iloc[48:]
    
    # Разбиение данных
    predict_may_start = '2016-04-30 23:00'
    predict_may_stop = '2016-05-31 23:00'
    precit_june_stop = '2016-06-30 23:00' 
    
    X_train = named_data.loc[:predict_may_start].drop(id_name, axis = 1)
    y_train = named_data.loc[:predict_may_start][id_name]
    
    X_val = named_data.loc[predict_may_start:predict_may_stop].drop(id_name, axis = 1)
    y_val = named_data.loc[predict_may_start:predict_may_stop][id_name]
    
    X_test = named_data.loc[predict_may_stop:].drop(id_name, axis = 1)
    y_test = named_data.loc[predict_may_stop:][id_name]
    
    #Строим Ridge модель с подбором гиперпараметра
    model = linear_model.Ridge()
    param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
    clf = GridSearchCV(model, param_grid, cv = 5)
    best_model = clf.fit(X_train, y_train)
    
    # Сохраним результаты разбиения данных, моделей и их прогнозы по словарям. Ключлм будет являться id региона
    linear_models_dict[id_name] = best_model
    X_val_dict[id_name] = X_val
    y_val_dict[id_name] = y_val
    y_val_pred_dict[id_name] = best_model.predict(X_val)
    X_test_dict[id_name] = X_test
    y_test_dict[id_name] = y_test
    y_test_pred_dict[id_name] = best_model.predict(X_test)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 Wall time: 3min 19s


In [25]:
%%time
Q_may_f = 0
for key in filtered_data.columns:
    print(key,end=' ')
    for date in may_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_val_pred_dict.get(key), index = y_val_dict.get(key).index, columns = ['val'])
            real_values = y_val_dict.get(key)
            Q_may_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_may = float(Q_may_f/(102*739*6))
print() 
print('Q_may: ', Q_may)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_may:  0.0208401952524313
Wall time: 2min 11s


In [26]:
# **Анализ предсказаний за июнь**

june_time_range = pd.date_range('2016-05-31 23:00', '2016-06-30 17:00', freq = 'H')

In [28]:
%%time
Q_june_f = 0
for key in filtered_data.columns:
    print(key,end=' ')
    for date in june_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_test_pred_dict.get(key), index = y_test_dict.get(key).index, columns = ['val'])
            real_values = y_test_dict.get(key)
            Q_june_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_june = float(Q_june_f/(102*739*6))
print()
print('Q_june: ', Q_june)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_june:  0.01973163209361697
Wall time: 2min 7s


In [30]:
%time
Q_june_f = 0
title_list = []
prediction_list = []
for key in filtered_data.columns:
    print(key,end=' ')
    for date in june_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_test_pred_dict.get(key), index = y_test_dict.get(key).index, columns = ['val'])
            real_values = y_test_dict.get(key)
                
            prediction_list.append(float(forecast.loc[date + timedelta(hours = step)]))
            title_list.append(str(key) + '_' + str(date)[:10]+ '_' + str(date.hour) + '_' + str(step))  
            
            Q_june_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_june = float(Q_june_f/(102*739*6))

Wall time: 0 ns
1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 

In [31]:
print(Q_june)

0.01973163209361697


In [32]:
kaggle_week6_df = pd.DataFrame(prediction_list, index = title_list, columns = ['y'])
kaggle_week6_df.index.name = 'id'
print(kaggle_week6_df.shape)
kaggle_week6_df.head()

(437580, 1)


,y
id,
1075_2016-05-31_23_1,25.990564
1075_2016-05-31_23_2,13.994253
1075_2016-05-31_23_3,4.993439
1075_2016-05-31_23_4,1.999449
1075_2016-05-31_23_5,1.000350


In [33]:
kaggle_week6_df.to_csv('kdw6.csv')

In [ ]:
# Ссылка:https://inclass.kaggle.com/c/yellowtaxi/leaderboard